In [1]:
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn import tree
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from collections import defaultdict

# we don't like warnings
# you can comment the following 2 lines if you'd like to
import warnings
warnings.filterwarnings('ignore')

# Matplotlib forms basis for visualization in Python
import matplotlib.pyplot as plt

# We will use the Seaborn library
import seaborn as sns
sns.set()

# Graphics in SVG format are more sharp and legible
%config InlineBackend.figure_format = 'svg'

In [2]:
df = pd.read_csv("mHealth_subject3.log", header=None, delim_whitespace=True)

df=df[df[23] != 0]
df=df[df[23] != 6]
df=df[df[23] != 7]
df=df[df[23] != 8]
df=df[df[23] != 12]

df.shape

def extractECG(dataFrame,lead=1):
    if lead == 1:
        lead = 4
    else :
        lead = 5
        
    data = dataFrame[lead]
    data = data.reset_index()
    data.columns=["index","readings"]
    return data['readings']
data=extractECG(df)

readings=25
def fmax(label_count):
    M=0
    for i in range (int (len (label_count)) ):
        if label_count[i]>label_count[M]:
            M=i
    return M
labels=[]
label_index=0
label = df[23].values
label_count=[0,0,0,0,0,0,0,0]
for i in range(int(len(label))):
    if label[i]==1:
        label_count[0]=label_count[0]+1
    elif label[i]==2:
        label_count[1]=label_count[1]+1
    elif label[i]==3:
        label_count[2]=label_count[2]+1
    elif label[i]==4:
        label_count[3]=label_count[3]+1
    elif label[i]==5:
        label_count[4]=label_count[4]+1
    elif label[i]==9:
        label_count[5]=label_count[5]+1
    elif label[i]==10:
        label_count[6]=label_count[6]+1
    elif label[i]==11:
        label_count[7]=label_count[7]+1
    
    if(i%readings==readings-1):
        M=fmax(label_count)
        labels.append(M)
        label_count=[0,0,0,0,0,0,0,0,0,0,0,0,0]
        label_index=label_index+1
labels=np.array(labels)

data=data.values
for j in range(data.size%readings):
    data = np.delete(data, (j), axis=0)
data.size

data= data.reshape(int(data.size/readings),1,readings)


In [3]:
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.utils import to_categorical

Using TensorFlow backend.


In [4]:
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, to_categorical(labels), test_size = 0.20, random_state = 0)


In [5]:
# from sklearn.preprocessing import StandardScaler

# def scale_data(trainX, testX, standardize):

# 	# standardize
# 	if standardize:
# 		s = StandardScaler()
# 		s.fit(trainX)
# 		trainX = s.transform(trainX)
# 		testX = s.transform(testX)
# 	# reshape
# 	trainX = trainX.reshape((trainX.shape))
# 	testX = testX.reshape((testX.shape))
# 	return trainX, testX
# X_train=X_train.reshape(786,25)
# X_test=X_test.reshape(X_test.shape[0],25)
# trainX, testX = scale_data(X_train, X_test, True)

X_train.shape

(786, 1, 25)

In [7]:

from keras.models import Sequential
from keras import layers
from keras.optimizers import RMSprop

model7 = Sequential()
model7.add(layers.Flatten(input_shape=(X_train.shape[1], X_train.shape[-1])))
model7.add(layers.Dense(32, activation='relu'))
model7.add(layers.Dense(8))

model7.compile(optimizer=RMSprop(), loss='mae')
history = model7.fit(X_train,y_train,steps_per_epoch=20,epochs=20)
                              

Instructions for updating:
Use tf.cast instead.
Epoch 1/20
20/20 [==============================] - 1s 52ms/step - loss: 0.2852
Epoch 2/20
20/20 [==============================] - 0s 3ms/step - loss: 0.2283
Epoch 3/20
20/20 [==============================] - 0s 2ms/step - loss: 0.1991
Epoch 4/20
20/20 [==============================] - 0s 3ms/step - loss: 0.1791
Epoch 5/20
20/20 [==============================] - 0s 5ms/step - loss: 0.1651
Epoch 6/20
20/20 [==============================] - 0s 3ms/step - loss: 0.1561
Epoch 7/20
20/20 [==============================] - 0s 2ms/step - loss: 0.1501
Epoch 8/20
20/20 [==============================] - 0s 3ms/step - loss: 0.1457
Epoch 9/20
20/20 [==============================] - 0s 3ms/step - loss: 0.1426
Epoch 10/20
20/20 [==============================] - 0s 4ms/step - loss: 0.1401
Epoch 11/20
20/20 [==============================] - 0s 3ms/step - loss: 0.1382
Epoch 12/20
20/20 [==============================] - 0s 3ms/step - loss: 0.1366


In [8]:
accuracy = model7.evaluate(X_test, y_test, batch_size=32, verbose=1)
print (accuracy)

197/197 [==============================] - 0s 1ms/step
0.13677601690219743


In [15]:
model = Sequential()
model.add(layers.GRU(32,
                     dropout=0.1,
                     recurrent_dropout=0.5,
                     return_sequences=True,
                     input_shape=(None, X_train.shape[-1])))
model.add(layers.GRU(64, activation='relu',
                     dropout=0.1, 
                     recurrent_dropout=0.3))

model.add(layers.Dense(8))

model.compile(optimizer=RMSprop(), loss='mae')

In [16]:
history = model.fit(X_train,y_train, epochs=40,verbose=1)

Epoch 1/40
786/786 [==============================] - 3s 4ms/step - loss: 0.1378
Epoch 2/40
786/786 [==============================] - 0s 201us/step - loss: 0.1325
Epoch 3/40
786/786 [==============================] - 0s 319us/step - loss: 0.1306
Epoch 4/40
786/786 [==============================] - 0s 264us/step - loss: 0.1294
Epoch 5/40
786/786 [==============================] - 0s 197us/step - loss: 0.1286
Epoch 6/40
786/786 [==============================] - 0s 189us/step - loss: 0.1279
Epoch 7/40
786/786 [==============================] - 0s 200us/step - loss: 0.1273
Epoch 8/40
786/786 [==============================] - 0s 247us/step - loss: 0.1267
Epoch 9/40
786/786 [==============================] - 0s 215us/step - loss: 0.1264
Epoch 10/40
786/786 [==============================] - 0s 247us/step - loss: 0.1260
Epoch 11/40
786/786 [==============================] - 0s 203us/step - loss: 0.1256
Epoch 12/40
786/786 [==============================] - 0s 207us/step - loss: 0.1253
Epo

In [17]:
accuracy = model.evaluate(X_test, y_test, batch_size=32, verbose=1)
print (accuracy)

197/197 [==============================] - 1s 4ms/step
0.1227173149434443
